# COMBINE AND MERGE DATASETS

In [1]:
#import sqlite3
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

## MERGE DATAFRAMES

In [2]:
df1 = DataFrame({'key': ['b','b','a','c','a','a','b'],
                 'data1':range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2 = DataFrame({'key': ['a','b','d'],
                 'data2':range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [4]:
'''
With just one and the same key, that key is the default 
pd.merge(df1, df2) is the same as 
   pd.merge(df1, df2, on='key') and 
   pd.merge(df1, df2, how='inner')
Note how keys c and d and their values is not part of the frame:
''' 
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [6]:
df3 = DataFrame({'lkey': ['b','b','a','c','a','a','b'],
                 'ldata':range(7)})

df4 = DataFrame({'rkey': ['a','b','d'],
                 'rdata':range(3)})

In [7]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')

,lkey,ldata,rkey,rdata
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [8]:
df5 = DataFrame({'key1': ['foo','foo','bar'],
                 'key2': ['one','two','one'],
                 'ldata': [1,2,3]})

df6 = DataFrame({'key1': ['foo','foo','bar','bar'],
                 'key2': ['one','two','one','two'],
                 'rdata': [4,5,6,7]})
df5

,key1,key2,ldata
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [9]:
df6

,key1,key2,rdata
0,foo,one,4
1,foo,two,5
2,bar,one,6
3,bar,two,7


In [10]:
pd.merge(df5, df6, on=['key1', 'key2'], how='outer')

,key1,key2,ldata,rdata
0,foo,one,1.0,4
1,foo,two,2.0,5
2,bar,one,3.0,6
3,bar,two,NaN,7


In [11]:
# observe this
pd.merge(df5, df6, on='key1')

,key1,key2_x,ldata,key2_y,rdata
0,foo,one,1,one,4
1,foo,one,1,two,5
2,foo,two,2,one,4
3,foo,two,2,two,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## MERGING ON INDEX 

In [12]:
df7 = DataFrame({'key': ['b','b','a','c','a','a'],
                 'value': range(6)})

df8 = DataFrame({'group_val': [3.5, 7]},
                 index = ['a','b'])

df7

,key,value
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5


In [13]:
df8

,group_val
a,3.5
b,7.0


In [14]:
pd.merge(df7, df8, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,b,0,7.0
1,b,1,7.0
2,a,2,3.5
4,a,4,3.5
5,a,5,3.5
3,c,3,NaN


## MERGING ON INDEX: join()

In [15]:
df9 = DataFrame([[1.,2.],[3.,4.],[5.,6.]], index=['a','b','c'], columns=['Ohio','Nevada'])
df9

,Ohio,Nevada
a,1.0,2.0
b,3.0,4.0
c,5.0,6.0


In [16]:
df10 = DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14,]], index=['b','c','d','e'], columns=['Missouri','Alabama'])
df10

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [17]:
pd.merge(df9, df10, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,3.0,4.0,7.0,8.0
c,5.0,6.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,NaN,NaN,13.0,14.0


In [18]:
# less code :)
df9.join(df10, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,3.0,4.0,7.0,8.0
c,5.0,6.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,NaN,NaN,13.0,14.0


In [19]:
df11 = DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14,]], index=['a','c','e','f'], columns=['New York','Oregon'])
df11

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,13.0,14.0


In [20]:
df9.join([df10, df11], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,3.0,4.0,7.0,8.0,NaN,NaN
c,5.0,6.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,NaN,NaN,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,13.0,14.0


## CONCATENATING ALONG AXIS

### np.concatenate()

In [21]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [22]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [23]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

### pd.concat()

In [24]:
s1 = Series([0,1], index=['a','b'])
s2 = Series([2,3,4], index=['c','d','e'])
s3 = Series([5,6], index=['f','g'])

s3

f    5
g    6
dtype: int64

In [25]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [26]:
df1 = DataFrame(np.arange(6).reshape(3,2), index=['a','b','c'], columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [27]:
df2 = DataFrame(np.arange(4).reshape(2,2), index=['a','b'], columns=['three', 'four'])
df2

,three,four
a,0,1
b,2,3


In [28]:
pd.concat([df1, df2], axis=1, keys=['L1', 'L2'])

L1        L2     
  one two three four
a   0   1   0.0  1.0
b   2   3   2.0  3.0
c   4   5   NaN  NaN